# Python, Data, 2025

---

* [Agregace](),
* [Pivot tabulky](),
* [Časové řady](#Časové-řady),
    - [úvodní motivace](#Úvod),
    - [základní objekty](#Základní-objekty),
    - [indexování časovými údaji](#Indexování-s-časem),
    - [funkce to_datetime](#Funkce-TO_DATETIME),
    - [funkce date_range](#Funkce-DATE_RANGE),
    - [resampling](#Resampling),
    - [zpřesňování](#Zpřesňování)

---

## Agregace

---


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.HtBtNx6dodlNptXV1QGHWQHaG0%26pid%3DApi&f=1&ipt=3a4d1dfa14a495127b4ece5cf689f00ed0c9e0836853358412e18161ae7c505f&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>


*Seskupování* neboli **agregace** jsou procesy, které patří k základní efektivní analýze dat.

### Jednoduchá agregace

---

Přesto, že veškeré základní **statistické údaje** prakticky nabízí metoda `describe()`, můžeš ocenit, když stejnou statistiku můžeš aplikovat **na tebou vybrané objekty**.

Mezi nejjednodušší postupy, jak data analyzovat patří metody jako:
* `sum()`,
* `mean()`,
* `median()`,
* `min()`,
* `max()`.

Všechny tyto metody umožní získat jedno samotné číslo, které ti umožní prohlédnout podstatu zadaného datasetu.

In [1]:
from pandas import read_csv, DataFrame

In [2]:
nemovitosti_df = read_csv("../onsite/housing.csv")

In [3]:
nemovitosti_df.head()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade
0,1,221900.0,3,1.00,1180,5650,1,3,7
1,2,538000.0,3,2.25,2570,7242,2,3,7
2,3,180000.0,2,1.00,770,10000,1,3,6
3,4,604000.0,4,3.00,1960,5000,1,5,7
4,5,510000.0,3,2.00,1680,8080,1,3,8


In [4]:
nemovitosti_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           10 non-null     int64  
 1   price        10 non-null     float64
 2   bedrooms     10 non-null     int64  
 3   bathrooms    10 non-null     float64
 4   sqft_living  10 non-null     int64  
 5   sqft_lot     10 non-null     int64  
 6   floors       10 non-null     int64  
 7   condition    10 non-null     int64  
 8   grade        10 non-null     int64  
dtypes: float64(2), int64(7)
memory usage: 848.0 bytes


In [5]:
nemovitosti_df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade
count,10.00000,1.000000e+01,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,5.50000,4.380750e+05,3.100000,2.100000,2002.500000,16846.200000,1.300000,3.200000,7.400000
std,3.02765,3.137603e+05,0.567646,1.094177,1305.367658,29937.105556,0.483046,0.632456,1.349897
min,1.00000,1.800000e+05,2.000000,1.000000,770.000000,5000.000000,1.000000,3.000000,6.000000
25%,3.25000,2.365000e+05,3.000000,1.125000,1305.000000,6624.750000,1.000000,3.000000,7.000000
50%,5.50000,3.074250e+05,3.000000,2.125000,1747.500000,7356.000000,1.000000,3.000000,7.000000
75%,7.75000,5.310000e+05,3.000000,2.437500,1942.500000,9303.250000,1.750000,3.000000,7.000000
max,10.00000,1.225000e+06,4.000000,4.500000,5420.000000,101930.000000,2.000000,5.000000,11.000000


<br>

Prvně vyberu sloupec, se kterým chci pracovat:

In [6]:
nemovitosti_df.loc[:, "price"]

0     221900.0
1     538000.0
2     180000.0
3     604000.0
4     510000.0
5    1225000.0
6     257500.0
7     291850.0
8     229500.0
9     323000.0
Name: price, dtype: float64

In [7]:
nemovitosti_df.loc[:, "price"].sum()

4380750.0

<br>

Často ale není dostačující, prozkoumat data pouze jednoduchých agregačních funkcí.

Další operace, které je potřeba pochopit jsou seskupování dat podle zadaných parametrů.

<br>

### Složitější agregace, seskupování GROUPBY

---

Metoda `groupby`, původně operace z SQL jazyka, je v rámci knihovny `pandas` všestraný pomocník pro seskupování dat na základě různých kritérií.

In [8]:
from pandas import DataFrame

In [9]:
pokus_s_cisly_df = DataFrame(
    {'klíč': ['A', 'B', 'C', 'A', 'B', 'C'],
     'číselná hodnota': range(6)},
)

In [10]:
pokus_s_cisly_df

,klíč,číselná hodnota
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


#### Klasické seskupení podle sloupečku

---

In [11]:
# pokus_s_cisly_df.groupby?

In [12]:
pokus_s_cisly_df.groupby("klíč")

Jde opět o tzv. *lazy evaluation* proces, samotný nic neprovede, pouze čeká na pokyn uživatele, který samotnou agregaci spustí.

In [13]:
type(pokus_s_cisly_df.groupby("klíč"))

pandas.core.groupby.generic.DataFrameGroupBy

Metoda standardně vrací `DataFrameGroupBy`.

In [14]:
pokus_s_cisly_df

,klíč,číselná hodnota
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [15]:
pokus_s_cisly_df.groupby("klíč").max()
# max_pokus_s_cisly
# count_pokus_s_cisly

,číselná hodnota
klíč,
A,3
B,4
C,5


Metoda `sum()` je pouze jednou z možností, se kterou můžeš pracovat.

<br>

#### Sloupečkové označování

---

Stejně jako `DataFrame` můžeš označovat také *GroupBy* objekty.

In [16]:
pokus_s_cisly_df

,klíč,číselná hodnota
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [17]:
pokus_s_cisly_df.groupby("klíč")

In [18]:
pokus_s_cisly_df.groupby("klíč")["číselná hodnota"]

In [19]:
pokus_s_cisly_df.groupby("klíč")["číselná hodnota"].sum()

klíč
A    3
B    5
C    7
Name: číselná hodnota, dtype: int64

<br>

V uplynulé ukázce je zadaná seskupování podle sloupečku `klíč`.

Dále je vybraný pouze konkrétní sloupeček, na který chceš spustit metodu `sum`.

<br>

Pokud potřebuješ nad vybraným objektem provádět některé procesy ručně, můžeš přes *GroupBy* objekt **iterovat**:

In [20]:
for (klic, hodnota) in pokus_s_cisly_df.groupby("klíč"):
    print(f"Klic: {klic}; Hodnota={hodnota}")

Klic: A; Hodnota=  klíč  číselná hodnota
0    A                0
3    A                3
Klic: B; Hodnota=  klíč  číselná hodnota
1    B                1
4    B                4
Klic: C; Hodnota=  klíč  číselná hodnota
2    C                2
5    C                5


<br>

### Agregace s metodami

---

Kromě jednoduchých agregací, nabízí *GroupBy* řadu další funkcionality.

Jde o metody:
* `aggregate`,
* `filter`,
* `transform`,
* `apply`.

#### Metoda AGGREGATE

---

In [21]:
import numpy

In [22]:
rng = numpy.random.RandomState(0)

In [23]:
from pandas import DataFrame

In [24]:
pokus_s_cisly_df = DataFrame(
    {'klíč': ['A', 'B', 'C', 'A', 'B', 'C', 'C'],
     'data_1': range(7),
     'data_2': rng.randint(0, 10, 7)}
)

In [25]:
pokus_s_cisly_df

,klíč,data_1,data_2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
6,C,6,3


Statistické metody určitě nabízí spousty pomůcek.

Objekt typu *GroupBy* umí pracovat také s funkcemi, stringy a celými listy.

In [26]:
pokus_s_cisly_df.groupby("klíč").aggregate("sum")  # .sum()

,data_1,data_2
klíč,,
A,3,8
B,5,7
C,13,15


In [27]:
pokus_s_cisly_df.groupby("klíč").aggregate(numpy.median)

,data_1,data_2
klíč,,
A,1.5,4.0
B,2.5,3.5
C,5.0,3.0


In [28]:
pokus_s_cisly_df.groupby("klíč").aggregate(["min", numpy.median, max])

data_1            data_2           
        min median max    min median max
klíč                                    
A         0    1.5   3      3    4.0   5
B         1    2.5   4      0    3.5   7
C         2    5.0   6      3    3.0   9

<br>

Metoda `aggregate`umožní vytvořit tzv. *podsloupce*.

Také umožní elegantně spojit různé matematické funkce (př. `min`, `"min"`, `numpy.min`).

<br>

Pokud neuložím nově seskupenou tabulku/sloupec, potom data neuchovám.

Metoda `groupby` **neupravuje zdrojová data**.

In [29]:
pokus_s_cisly_df

,klíč,data_1,data_2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
6,C,6,3


Můžeš říct, že metoda `aggregate` se používá k aplikaci **jedné nebo více agregačních funkcí na seskupená data**.

| pobocka | ovoce | prodane_mnozstvi | cena |
| :- | :- | :- | :- |
| A | apple | 10 | 20 |
| A | banana | 15 | 12 |
| B | apple | 8 | 22 |
| B | banana | 30 | 10 |
| C | apple | 20 | 18 |
| C | banana | 25 | 15 |

In [30]:
ovoce_data = {
    'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
    'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
    'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
    'cena': [20, 12, 22, 10, 18, 15]
}

In [31]:
ovoce_data.keys()

dict_keys(['pobocka', 'ovoce', 'prodane_mnozstvi', 'cena'])

In [32]:
ovoce_df = DataFrame(ovoce_data)

In [33]:
ovoce_df.head(6)

,pobocka,ovoce,prodane_mnozstvi,cena
0,A,jablko,10,20
1,A,banan,15,12
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


In [34]:
ovoce_df.groupby('pobocka').aggregate({'prodane_mnozstvi': ["min", numpy.median, max]})

prodane_mnozstvi           
                     min median max
pobocka                            
A                     10   12.5  15
B                      8   19.0  30
C                     20   22.5  25

Pro jednodušší analýzu stačí vybrat sloupec:

In [35]:
ovoce_df.groupby('pobocka')['prodane_mnozstvi'].sum()

pobocka
A    25
B    38
C    45
Name: prodane_mnozstvi, dtype: int64

<br>

Je nutné vždy ověřit výstup po agregaci, jestli jsem správně vybral metody:

In [36]:
ovoce_df.groupby('pobocka').sum()

,ovoce,prodane_mnozstvi,cena
pobocka,,,
A,jablkobanan,25,32
B,jablkobanan,38,32
C,jablkobanan,45,33


In [37]:
ovoce_df.groupby('pobocka').aggregate({'prodane_mnozstvi': sum, 'cena': 'mean'})

,prodane_mnozstvi,cena
pobocka,,
A,25,16.0
B,38,16.0
C,45,16.5


1. Nejprve jsou hodnoty seskupení podle sloupce `pobocka`,
2. poté specifikuješ pomocí `aggregate` funkce a sloupce,
3. .. tedy sumarizovat hodnoty v `prodane_mnozstvi` a získat průměr `cena` pro každou pobočku.

### Metoda FILTER

---

Filtrování ti umožní zahodit takové údaje, které nesplňují zadanou podmínku.

Metoda `filter` se používá k vybrání **seskupených dat** podle splnění **určité podmínky**.

Představme si, že máme následující dataset s informacemi o prodeji ovoce v různých obchodech:

```
pobocka,ovoce,prodane_mnozstvi,cena
A,apple,10,20
A,banana,15,12
B,apple,8,22
B,banana,30,10
C,apple,20,18
C,banana,25,15
```

In [38]:
ovoce_data = {
    'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
    'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
    'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
    'cena': [20, 12, 22, 10, 18, 15]
}

In [39]:
ovoce_df = DataFrame(ovoce_data)

In [40]:
ovoce_df

,pobocka,ovoce,prodane_mnozstvi,cena
0,A,jablko,10,20
1,A,banan,15,12
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


In [41]:
    def vyber_s_prodejem_nad_limit(data, limit=30):
        return data['prodane_mnozstvi'].sum() > limit

<br>

Funkce `vyber_s_prodejem_nad_limit`, tedy **filtrovací funkce**, musí vracet **boolean** datový typ.

In [42]:
vyfiltrovane_df = ovoce_df.groupby('pobocka').filter(vyber_s_prodejem_nad_limit)

In [43]:
vyfiltrovane_df.groupby('pobocka')['prodane_mnozstvi'].sum()

pobocka
B    38
C    45
Name: prodane_mnozstvi, dtype: int64

1. Nejprve seskupíš data podle sloupce `pobocka` pomocí `groupby`,
2. dále použiješ metodu `filter`, která umožňuje použít uživatelem definovanou funkci (`vyber_pouze_pobocky_s_prodejem_nad_limit`),
3. definuješ funkci, která vybere pouze pobočky s větším prodejem než je parametr `limit`,
4. metoda `filter` pak vybere pouze ty řádky, které splňují tuto podmínku.

In [44]:
ovoce_df[ovoce_df['prodane_mnozstvi'] > 19]  # Boolean porovnávání

,pobocka,ovoce,prodane_mnozstvi,cena
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


### Metoda TRANSFORM

---

Zatímco předchozí výsledky *agregace* pomocí `groupby` vraceli redukované množství dat.

Transformace obvykle vrací data o stejném rozsahu jako vstupní data. Jenom upravená.

Metoda `transform` se používá k aplikaci určité **transformační funkce** na každý prvek seskupených dat.

In [45]:
ovoce_df

,pobocka,ovoce,prodane_mnozstvi,cena
0,A,jablko,10,20
1,A,banan,15,12
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


In [46]:
def vrat_procenta_z_celkoveho_prodeje(udaj):
    return round(udaj / udaj.sum() * 100, 1)

<br>

Vytvoření nového sloupečku `vrat_procenta_z_celkoveho_prodeje`:

In [47]:
ovoce_df['Procento z celkoveho prodeje'] = \
    ovoce_df.groupby('pobocka')['prodane_mnozstvi'].transform(
        vrat_procenta_z_celkoveho_prodeje
    )

In [48]:
# ovoce_df

1. Nejprve seskupíš data podle sloupce `pobocka` pomocí `groupby`,
2. dále použiješ metodu `transform`, která umožňuje použít uživatelem definovanou funkci,
3. definuješ funkci, která vybere vypočítá procentuální vyjádření prodaného zboží pro pobočku,
4. přidáš nový sloupeček `'Procento z celkoveho prodeje'`.

### Metoda `APPLY`

---

Metoda `apply` ti také dovolí, používat uživatelem definované funkce na seskupená data.

Následně vrací objekt knihovny `pandas` (buď `DataFrame`, nebo `Series`, a nebo skalární hodnota).

In [49]:
ovoce_df

,pobocka,ovoce,prodane_mnozstvi,cena,Procento z celkoveho prodeje
0,A,jablko,10,20,40.0
1,A,banan,15,12,60.0
2,B,jablko,8,22,21.1
3,B,banan,30,10,78.9
4,C,jablko,20,18,44.4
5,C,banan,25,15,55.6


In [50]:
def vypocitej_vydelek_za_artikl(seskupene):
    seskupene['vydelek_za_artikl'] = seskupene['prodane_mnozstvi'] * seskupene['cena']
    return seskupene

<br>

Metoda `apply` je **obecnější a flexibilnější** než `transform`.

`apply` umožňuje použít uživatelem definovanou funkci na každou skupinu po seskupení dat pomocí `groupby`.

Výsledek metody `apply` může mít jiný tvar než původní data.

In [51]:
ovoce_df.groupby('pobocka').apply(vypocitej_vydelek_za_artikl)

pobocka   ovoce  prodane_mnozstvi  cena  \
pobocka                                             
A       0       A  jablko                10    20   
        1       A   banan                15    12   
B       2       B  jablko                 8    22   
        3       B   banan                30    10   
C       4       C  jablko                20    18   
        5       C   banan                25    15   

           Procento z celkoveho prodeje  vydelek_za_artikl  
pobocka                                                     
A       0                          40.0                200  
        1                          60.0                180  
B       2                          21.1                176  
        3                          78.9                300  
C       4                          44.4                360  
        5                          55.6                375

In [52]:
ovoce_df.groupby('pobocka', group_keys=False).apply(vypocitej_vydelek_za_artikl)

,pobocka,ovoce,prodane_mnozstvi,cena,Procento z celkoveho prodeje,vydelek_za_artikl
0,A,jablko,10,20,40.0,200
1,A,banan,15,12,60.0,180
2,B,jablko,8,22,21.1,176
3,B,banan,30,10,78.9,300
4,C,jablko,20,18,44.4,360
5,C,banan,25,15,55.6,375


In [53]:
# ovoce_df.groupby?

Od posledních verzí frameworku (`1.5.0` a vyšší) platí, že pokud bude výsledkem `DataFrame` nebo `Series` musíš uvést argument pro `group_keys=True`).

<br>

In [54]:
vystup = ovoce_df.groupby('pobocka', group_keys=True).apply(vypocitej_vydelek_za_artikl)

In [55]:
vystup

pobocka   ovoce  prodane_mnozstvi  cena  \
pobocka                                             
A       0       A  jablko                10    20   
        1       A   banan                15    12   
B       2       B  jablko                 8    22   
        3       B   banan                30    10   
C       4       C  jablko                20    18   
        5       C   banan                25    15   

           Procento z celkoveho prodeje  vydelek_za_artikl  
pobocka                                                     
A       0                          40.0                200  
        1                          60.0                180  
B       2                          21.1                176  
        3                          78.9                300  
C       4                          44.4                360  
        5                          55.6                375

In [56]:
ovoce_df

,pobocka,ovoce,prodane_mnozstvi,cena,Procento z celkoveho prodeje
0,A,jablko,10,20,40.0
1,A,banan,15,12,60.0
2,B,jablko,8,22,21.1
3,B,banan,30,10,78.9
4,C,jablko,20,18,44.4
5,C,banan,25,15,55.6


Na první pohled vypadají metody `apply` a `transform` docela podobně.

Metoda `transform` je trochu omezenější než `apply`.

Slouží k aplikaci uživatelem definované nebo vestavěné funkce **na každý prvek** skupiny po seskupení s `groupby`.

`transform` musí vracet hodnotu stejného tvaru jako vstupní data.

### Souhrn k metodám APPLY a TRANSFORM

---

Výsledek metody `transform` **má stejný tvar jako původní data**.

#### Apply
* potřebuješ výsledek, který **má jiný tvar než původní data**,
* umí zpracovat **více sloupečků současně**.

#### Transform
* potřebuješ výsledek, který **má stejný tvar jako původní data**,
* umí zpracovat **pouze jeden sloupeček**.

In [57]:
df_rozdily = DataFrame({
    'KLIC': ['A','B','C'] * 3,
    'A': numpy.arange(9),
    'B': [1,2,3] * 3,
})

In [58]:
df_rozdily

,KLIC,A,B
0,A,0,1
1,B,1,2
2,C,2,3
3,A,3,1
4,B,4,2
5,C,5,3
6,A,6,1
7,B,7,2
8,C,8,3


#### `transform` vrací výsledky ve stejném tvaru

---

In [59]:
def vypocitej_sumu(data):
    return data.sum()

In [60]:
seskup_df_rozdily_apply = df_rozdily.groupby('KLIC')['A'].apply(vypocitej_sumu)

In [61]:
seskup_df_rozdily_apply

KLIC
A     9
B    12
C    15
Name: A, dtype: int64

In [62]:
seskup_df_rozdily_trans = df_rozdily.groupby('KLIC')['A'].transform(vypocitej_sumu)

In [63]:
seskup_df_rozdily_trans

0     9
1    12
2    15
3     9
4    12
5    15
6     9
7    12
8    15
Name: A, dtype: int64

#### `apply` umí pracovat s více sloupečky, `transform` jen s jedním

---

In [64]:
df_rozdily

,KLIC,A,B
0,A,0,1
1,B,1,2
2,C,2,3
3,A,3,1
4,B,4,2
5,C,5,3
6,A,6,1
7,B,7,2
8,C,8,3


In [65]:
def vypocitej_rozdil(data):
    return data['B'] - data['A']

In [66]:
df_rozdily.groupby('KLIC').apply(vypocitej_rozdil)

KLIC   
A     0    1
      3   -2
      6   -5
B     1    1
      4   -2
      7   -5
C     2    1
      5   -2
      8   -5
dtype: int64

In [67]:
# df_rozdily.groupby('KLIC').transform(vypocitej_rozdil)

---

<br>

**🧠 CVIČENÍ 🧠, procvič si funkcí GroupBy a agregační funkce**

Máš zadaný takový datový set.

Následně:
1. Pomocí metody `filter` vyber prodejny, které prodaly **alespoň 30 produktů**,
2. na filtrovaném datasetu použijte metodu `apply` pro výpočet **celkového příjmu z prodeje pro každý obchod**.

In [68]:
from pandas import DataFrame

In [69]:
df_prodej_hardware = DataFrame({
    'prodejna_id': (5, 4, 1, 5, 5, 1, 4, 2, 5, 1, 3, 1, 3, 4, 2, 1, 5, 4, 1, 5),
    'transakce_id': (1278, 1216, 1866, 1872, 1797, 1272, 1880, 1061, 1595, 1879, 1728,
       1341, 1396, 1698, 1018, 1176, 1611, 1395, 1444, 1232),
    'predmet_prodeje': ('grafická_karta', 'SSD', 'RAM', 'procesor', 'grafická_karta',
       'základní_deska', 'SSD', 'SSD', 'grafická_karta', 'RAM',
       'grafická_karta', 'procesor', 'grafická_karta', 'SSD',
       'grafická_karta', 'RAM', 'základní_deska', 'HDD', 'grafická_karta',
       'RAM'),
    'pocet_prodanych_ks': (1,  5,  6,  6,  3,  7,  9, 10,  8,  6,  8,  5,  8, 10,  4, 10,  8,
       10,  2,  5),
    'cena_predmetu': (19500.69874949, 19731.10951735, 14114.15342339, 10953.87914371,
        6535.78851758, 16369.00288429, 13852.2578648 ,  3671.03031723,
       18263.08009763, 16539.476237  , 19021.09830919, 14651.53041357,
       12461.59632075,  8655.73920767, 18688.2054254 , 17388.24584526,
        1381.76406707,  1014.1560027 ,  7841.03565412, 16305.78995025)
})

In [70]:
df_prodej_hardware.head(7)

,prodejna_id,transakce_id,predmet_prodeje,pocet_prodanych_ks,cena_predmetu
0,5,1278,grafická_karta,1,19500.698749
1,4,1216,SSD,5,19731.109517
2,1,1866,RAM,6,14114.153423
3,5,1872,procesor,6,10953.879144
4,5,1797,grafická_karta,3,6535.788518
5,1,1272,základní_deska,7,16369.002884
6,4,1880,SSD,9,13852.257865


In [71]:
df_prodej_hardware.groupby('prodejna_id')['pocet_prodanych_ks'].sum()

prodejna_id
1    36
2    14
3    16
4    34
5    31
Name: pocet_prodanych_ks, dtype: int64

In [72]:
# S filtrem:
def vyber_prodejny_s_prodejem_pres_tricet_ks(data_prodej,
                                             jmeno_sl='pocet_prodanych_ks',
                                             limit=30):
    if jmeno_sl not in data_prodej.columns:
        return None
    else:
        return data_prodej[jmeno_sl].sum() > limit

In [73]:
vyfiltr_df = df_prodej_hardware.groupby('prodejna_id').filter(vyber_prodejny_s_prodejem_pres_tricet_ks)

In [74]:
vyfiltr_df.head()

,prodejna_id,transakce_id,predmet_prodeje,pocet_prodanych_ks,cena_predmetu
0,5,1278,grafická_karta,1,19500.698749
1,4,1216,SSD,5,19731.109517
2,1,1866,RAM,6,14114.153423
3,5,1872,procesor,6,10953.879144
4,5,1797,grafická_karta,3,6535.788518


In [75]:
vyfiltr_df['predmet_prodeje'].unique()

array(['grafická_karta', 'SSD', 'RAM', 'procesor', 'základní_deska',
       'HDD'], dtype=object)

In [76]:
vyfiltr_df.groupby('prodejna_id')

In [77]:
df_prodej_hardware.head(2)

,prodejna_id,transakce_id,predmet_prodeje,pocet_prodanych_ks,cena_predmetu
0,5,1278,grafická_karta,1,19500.698749
1,4,1216,SSD,5,19731.109517


In [78]:
# Bez filtru:

In [79]:
seskupene_prodeje = df_prodej_hardware.groupby('prodejna_id')['pocet_prodanych_ks'].sum()

In [80]:
type(seskupene_prodeje)

pandas.core.series.Series

In [81]:
seskupene_prodeje[seskupene_prodeje > 30]

prodejna_id
1    36
4    34
5    31
Name: pocet_prodanych_ks, dtype: int64

In [82]:
def vypocti_celkovy_prijem_obchodu(df_prodej_hardware):
    return round(df_prodej_hardware['pocet_prodanych_ks']
                 * df_prodej_hardware['cena_predmetu'], 2)

In [83]:
vyfiltr_df['Celkové prodeje'] = \
    vyfiltr_df.groupby('prodejna_id', group_keys=False).apply(vypocti_celkovy_prijem_obchodu)

In [84]:
type(vyfiltr_df.groupby('prodejna_id')['Celkové prodeje'].sum())

pandas.core.series.Series

<br>

Pokud bude potřebné, seřadit sestupně podle celkového výdělku prodejny:

In [85]:
vyfiltr_df \
    .groupby('prodejna_id')['Celkové prodeje'] \
    .sum() \
    .sort_values(ascending=False)

prodejna_id
1    561326.98
5    343519.04
4    320024.82
Name: Celkové prodeje, dtype: float64

<details>
    <summary>▶️ Řešení</summary>
    
```python
def vyber_prodejny_s_vice_nez_triceti_produkty(
    data: DataFrame, limit: int = 30
):
    return data['pocet_prodanych_ks'].sum() >= limit

hledane_prodejny = df_prodej_hardware.groupby('prodejna_id').filter(
    vyber_prodejny_s_vice_nez_triceti_produkty
)

agregovano = df_prodej_hardware.groupby('prodejna_id')['pocet_prodanych_ks'].sum()
odfiltrovano = agregovano[agregovano > 30]

print(odfiltrovano)
```
</details>

---


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse2.mm.bing.net%2Fth%3Fid%3DOIP.P16hnt14CM-mJ5eAKykxLAHaHa%26pid%3DApi&f=1&ipt=27e3f5b23fe44567ad0730192d5dcb13c7c23aa6e87579d16df05e27461dc12c&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

## Pivot tabulky

---


*Pivot tabulky* (*kontingenční tabulky*) jsou užitečné pro přehledné zobrazení a analýzu dat z tabulkových zdrojů.

Pomocí pivot tabulek můžeš **seskupit data podle určitých kategorií** a provést agregaci hodnot.

Nejprve si představ situaci bez pivot tabulek, pomocí ukázky níže:

In [86]:
# !pip install seaborn

In [87]:
import seaborn

<br>

V této ukazce použiješ vzorová data týkající se [nehody lodi Titanic](https://en.wikipedia.org/wiki/Sinking_of_the_Titanic):

In [88]:
df_titanic = seaborn.load_dataset('titanic')

In [89]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


<br>

Pro jakoukoliv pokročilou analýzu dat, potřebuješ údaj seskupit.

Tak jak jsi do prováděl doposud, pomocí `groupby`:

In [90]:
df_titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


<br>

Pokud potřebuješ identifikovat všechny unikátní hodnoty ve sloupci, vyber sloupec a použij metodu `unique()`:

In [91]:
df_titanic["sex"].unique()

array(['male', 'female'], dtype=object)

<br>

Pro následující rozbor seskupím data podle pohlaví:

In [92]:
df_titanic.groupby('sex')

<br>

Potřebuješ průměrnou hodnotu, ať je na sloupečku `survived` vidět poměr přeživších:

In [93]:
df_titanic.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


In [94]:
df_titanic.groupby('sex')['survived'].mean()

sex
female    0.742038
male      0.188908
Name: survived, dtype: float64

<br>

Pro lepší čitelnost **zaokrouhlím výsledek na dvě desetinné čísla**:

In [95]:
# round(df_titanic.)

In [96]:
df_titanic.groupby('sex')['survived'].mean().round(2)

sex
female    0.74
male      0.19
Name: survived, dtype: float64

Zaokrouhlit je možné nejenom pomocí Pythonu a funkce `round()` ale také metoda `round()`.

Takový průzkum z této studie ti dá jasný pohled na věc:
* 3 ze 4 žen přežily,
* 1 z 5 mužů přežil.

<br>

Pokud budeš potřebovat detailnější analýzy, budeš potřebovat více dat.

Třeba situaci, kde kromě pohlaví, bereš v potaz **třídu cestujících** `class`:

In [97]:
df_titanic.head(2)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False


In [98]:
df_titanic["class"].unique()

['Third', 'First', 'Second']
Categories (3, object): ['First', 'Second', 'Third']

In [99]:
type(df_titanic.groupby(['sex', 'class'])['survived'].mean())

pandas.core.series.Series

In [100]:
type(df_titanic.groupby(['sex', 'class'])[['survived']].mean())

pandas.core.frame.DataFrame

In [101]:
df_titanic.groupby(['sex', 'class'])[['survived']].mean()

survived
sex    class           
female First   0.968085
       Second  0.921053
       Third   0.500000
male   First   0.368852
       Second  0.157407
       Third   0.135447

In [102]:
df_titanic.groupby(['sex', 'class'])[['survived']].mean().round(2)

survived
sex    class           
female First       0.97
       Second      0.92
       Third       0.50
male   First       0.37
       Second      0.16
       Third       0.14

<br>

Metodou `unstack` si můžeš vytvořit nové sloupečky, které jsou postavené na novém Indexu, nebo Indexech (*Multiindex*):

In [103]:
df_titanic.groupby(['sex', 'class'])['survived'].mean().round(2).unstack()

class,First,Second,Third
sex,,,
female,0.97,0.92,0.50
male,0.37,0.16,0.14


Takový průzkum ti dá skutečně lepší pohled na věc.

Současně ale roste **náročnost ohlášení**.

Zápis "bobtná" a **stává se náročnějším na přečtení a pochopení**.

<br>

### Pivot tabulka

---

Podobné řešení ti nabízí funkce `pivot_table`:

In [104]:
df_titanic.groupby(['sex', 'class'])['survived'].mean().round(2).unstack()

class,First,Second,Third
sex,,,
female,0.97,0.92,0.50
male,0.37,0.16,0.14


In [105]:
df_titanic.pivot_table('survived', index='sex', columns='class').round(2)

class,First,Second,Third
sex,,,
female,0.97,0.92,0.50
male,0.37,0.16,0.14


In [106]:
pivot_df = df_titanic.pivot_table('survived', index='sex', columns='class').round(2)

In [107]:
df_titanic.index

RangeIndex(start=0, stop=891, step=1)

In [108]:
pivot_df.index

Index(['female', 'male'], dtype='object', name='sex')

Zásádním rozdílem je ovšem **čitelnost**, kterou máš pro tuto variantu zápisu.

```
Clarity beats purity
```

Pomocí vhodných argumentů, můžeš doplnit vysvětlivky tam, kde funkce `groupby` nemohla.

Stejně platí, že pokud budeš potřebovat **další Index**, můžeš si pomoci funkcí `cut`:

In [109]:
df_titanic.head(2)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False


In [110]:
df_titanic['age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: age, dtype: float64

In [111]:
from pandas import cut

In [112]:
age = cut(df_titanic['age'], [0, 40, 60, 70])

In [113]:
age  # < xxx , xxx )

0      (0.0, 40.0]
1      (0.0, 40.0]
2      (0.0, 40.0]
3      (0.0, 40.0]
4      (0.0, 40.0]
          ...     
886    (0.0, 40.0]
887    (0.0, 40.0]
888            NaN
889    (0.0, 40.0]
890    (0.0, 40.0]
Name: age, Length: 891, dtype: category
Categories (3, interval[int64, right]): [(0, 40] < (40, 60] < (60, 70]]

In [114]:
vystup_vek_df = df_titanic.pivot_table('survived',
                                       index=['sex', age],
                                       columns='class').round(2)

In [115]:
vystup_vek_df

class            First  Second  Third
sex    age                           
female (0, 40]    0.97    0.93   0.49
       (40, 60]   0.96    0.85   0.00
       (60, 70]   1.00     NaN   1.00
male   (0, 40]    0.51    0.17   0.16
       (40, 60]   0.34    0.06   0.07
       (60, 70]   0.00    0.33   0.00

In [116]:
vystup_vek_df.index

MultiIndex([('female',  (0, 40]),
            ('female', (40, 60]),
            ('female', (60, 70]),
            (  'male',  (0, 40]),
            (  'male', (40, 60]),
            (  'male', (60, 70])],
           names=['sex', 'age'])

Pomocí funkce `cut` doplníš tabulku na **MultiIndex**.

Ten nyní vytvoří i rozsah pro věkovou kategorii.

<br>

#### 🧠 CVIČENÍ 🧠, procvič si pivot tabulky

---

Ze zadaného datasetu vytvoř **pivot tabulku**, která zobrazí **počet prodaných kusů ovoce** pro **každý obchod** a **druh ovoce**.

In [117]:
ovoce_data = {
    'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
    'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
    'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
    'cena': [20, 12, 22, 10, 18, 15]
}

In [118]:
# pivot_table()
#              prodane_mnozstvi      jablko banan
# Pobocka
#    A 
#    B
#    C 

In [119]:
ovoce_data_df = DataFrame(ovoce_data)

In [120]:
ovoce_data_df['ovoce'].unique()

array(['jablko', 'banan'], dtype=object)

In [121]:
ovoce_data_df.head()

,pobocka,ovoce,prodane_mnozstvi,cena
0,A,jablko,10,20
1,A,banan,15,12
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18


In [122]:
ovoce_data_df.pivot_table('prodane_mnozstvi', index='pobocka', columns='ovoce')

ovoce,banan,jablko
pobocka,,
A,15,10
B,30,8
C,25,20


In [123]:
ovoce_data_df['cena'].min()

10

In [124]:
cenovy_intervaly = cut(ovoce_data_df['cena'],
                       [ovoce_data_df['cena'].min(),
                        15, ovoce_data_df['cena'].max()], include_lowest=True)

In [125]:
cenovy_intervaly

0     (15.0, 22.0]
1    (9.999, 15.0]
2     (15.0, 22.0]
3    (9.999, 15.0]
4     (15.0, 22.0]
5    (9.999, 15.0]
Name: cena, dtype: category
Categories (2, interval[float64, right]): [(9.999, 15.0] < (15.0, 22.0]]

In [126]:
ovoce_data_df.pivot_table('prodane_mnozstvi',
                          index=['pobocka', cenovy_intervaly], columns='ovoce')

ovoce                  banan  jablko
pobocka cena                        
A       (9.999, 15.0]   15.0     NaN
        (15.0, 22.0]     NaN    10.0
B       (9.999, 15.0]   30.0     NaN
        (15.0, 22.0]     NaN     8.0
C       (9.999, 15.0]   25.0     NaN
        (15.0, 22.0]     NaN    20.0

In [127]:
ovoce_data_df

,pobocka,ovoce,prodane_mnozstvi,cena
0,A,jablko,10,20
1,A,banan,15,12
2,B,jablko,8,22
3,B,banan,30,10
4,C,jablko,20,18
5,C,banan,25,15


<details>
    <summary>▶️ Řešení</summary>
    
```python
pivot_tabulka = df_ovoce.pivot_table(values="prodane_mnozstvi", index="pobocka", columns="ovoce")
```
</details>

In [128]:
# !pip install openpyxl

In [129]:
from pandas import read_excel

In [130]:
data_df = read_excel('../onsite/RESULT_LIST_ANONYM_CAPACITY_MARKET_FCR_2025-05-20_2025-05-20.xlsx')

/home/matous/projects/python-data/env/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [131]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1686 entries, 0 to 1685
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   DATE_FROM                          1686 non-null   datetime64[ns]
 1   DATE_TO                            1686 non-null   datetime64[ns]
 2   TYPE_OF_RESERVES                   1686 non-null   object        
 3   PRODUCT                            1686 non-null   object        
 4   OFFERED_CAPACITY_PRICE_[EUR/MW]    1686 non-null   float64       
 5   OFFERED_CAPACITY_[MW]              1686 non-null   int64         
 6   ALLOCATED_CAPACITY_[MW]            1686 non-null   int64         
 7   COUNTRY                            1686 non-null   object        
 8   SETTLEMENTCAPACITY_PRICE_[EUR/MW]  1686 non-null   float64       
 9   NOTE                               65 non-null     object        
dtypes: datetime64[ns](2), float64(2), in

In [132]:
data_df.head(3)

,DATE_FROM,DATE_TO,TYPE_OF_RESERVES,PRODUCT,OFFERED_CAPACITY_PRICE_[EUR/MW],OFFERED_CAPACITY_[MW],ALLOCATED_CAPACITY_[MW],COUNTRY,SETTLEMENTCAPACITY_PRICE_[EUR/MW],NOTE
0,2025-05-20,2025-05-20,FCR,NEGPOS_00_04,0.0,10,10,DE,22.0,NaN
1,2025-05-20,2025-05-20,FCR,NEGPOS_00_04,0.0,5,5,DE,22.0,NaN
2,2025-05-20,2025-05-20,FCR,NEGPOS_00_04,0.0,4,4,DE,22.0,NaN


#### 1. Převedeme data ve sloupcích

---

In [133]:
data_df.columns

Index(['DATE_FROM', 'DATE_TO', 'TYPE_OF_RESERVES', 'PRODUCT',
       'OFFERED_CAPACITY_PRICE_[EUR/MW]', 'OFFERED_CAPACITY_[MW]',
       'ALLOCATED_CAPACITY_[MW]', 'COUNTRY',
       'SETTLEMENTCAPACITY_PRICE_[EUR/MW]', 'NOTE'],
      dtype='object')

In [134]:
soucasne_poradi_sloupcu = tuple(data_df.columns)

In [135]:
soucasne_poradi_sloupcu

('DATE_FROM',
 'DATE_TO',
 'TYPE_OF_RESERVES',
 'PRODUCT',
 'OFFERED_CAPACITY_PRICE_[EUR/MW]',
 'OFFERED_CAPACITY_[MW]',
 'ALLOCATED_CAPACITY_[MW]',
 'COUNTRY',
 'SETTLEMENTCAPACITY_PRICE_[EUR/MW]',
 'NOTE')

In [136]:
nove_poradi_sloupcu = (
     'DATE_FROM',
     'DATE_TO',
     'TYPE_OF_RESERVES',
     'PRODUCT',
     'OFFERED_CAPACITY_PRICE_[EUR/MW]',
     'OFFERED_CAPACITY_PRICE_NEW',
     'OFFERED_CAPACITY_[MW]',
     'ALLOCATED_CAPACITY_[MW]',
     'COUNTRY',
     'SETTLEMENTCAPACITY_PRICE_[EUR/MW]',
     'SETTLEMENTCAPACITY_PRICE_NEW',
     'NOTE'
)

In [137]:
data_df['OFFERED_CAPACITY_PRICE_NEW'] = data_df['OFFERED_CAPACITY_PRICE_[EUR/MW]'] / 4

In [138]:
data_df['SETTLEMENTCAPACITY_PRICE_NEW'] = data_df['SETTLEMENTCAPACITY_PRICE_[EUR/MW]'] / 4

In [139]:
data_df.tail(2)

,DATE_FROM,DATE_TO,TYPE_OF_RESERVES,PRODUCT,OFFERED_CAPACITY_PRICE_[EUR/MW],OFFERED_CAPACITY_[MW],ALLOCATED_CAPACITY_[MW],COUNTRY,SETTLEMENTCAPACITY_PRICE_[EUR/MW],NOTE,OFFERED_CAPACITY_PRICE_NEW,SETTLEMENTCAPACITY_PRICE_NEW
1684,2025-05-20,2025-05-20,FCR,NEGPOS_20_24,57.60,1,1,CH,58.14,NaN,14.400,14.535
1685,2025-05-20,2025-05-20,FCR,NEGPOS_20_24,58.14,1,1,DE,58.14,NaN,14.535,14.535


<br>

Pro změnu pořadí sloupců, nachystáme novou tabulku, s novým pořadím:

In [140]:
from pandas import DataFrame

In [141]:
novy_df = DataFrame(data_df, columns=nove_poradi_sloupcu)

In [142]:
novy_df.tail(2)

,DATE_FROM,DATE_TO,TYPE_OF_RESERVES,PRODUCT,OFFERED_CAPACITY_PRICE_[EUR/MW],OFFERED_CAPACITY_PRICE_NEW,OFFERED_CAPACITY_[MW],ALLOCATED_CAPACITY_[MW],COUNTRY,SETTLEMENTCAPACITY_PRICE_[EUR/MW],SETTLEMENTCAPACITY_PRICE_NEW,NOTE
1684,2025-05-20,2025-05-20,FCR,NEGPOS_20_24,57.60,14.400,1,1,CH,58.14,14.535,NaN
1685,2025-05-20,2025-05-20,FCR,NEGPOS_20_24,58.14,14.535,1,1,DE,58.14,14.535,NaN


#### 2. Agregace podle některých sloupců

---

Necháme si vypsat unikátní hodnoty pro hledané sloupce:

In [143]:
novy_df['PRODUCT'].unique()

array(['NEGPOS_00_04', 'NEGPOS_04_08', 'NEGPOS_08_12', 'NEGPOS_12_16',
       'NEGPOS_16_20', 'NEGPOS_20_24'], dtype=object)

In [144]:
novy_df['COUNTRY'].unique()

array(['DE', 'BE', 'FR', 'NL', 'CZ', 'DK', 'CH', 'AT', 'SI'], dtype=object)

In [145]:
novy_df.head(2)

,DATE_FROM,DATE_TO,TYPE_OF_RESERVES,PRODUCT,OFFERED_CAPACITY_PRICE_[EUR/MW],OFFERED_CAPACITY_PRICE_NEW,OFFERED_CAPACITY_[MW],ALLOCATED_CAPACITY_[MW],COUNTRY,SETTLEMENTCAPACITY_PRICE_[EUR/MW],SETTLEMENTCAPACITY_PRICE_NEW,NOTE
0,2025-05-20,2025-05-20,FCR,NEGPOS_00_04,0.0,0.0,10,10,DE,22.0,5.5,NaN
1,2025-05-20,2025-05-20,FCR,NEGPOS_00_04,0.0,0.0,5,5,DE,22.0,5.5,NaN


In [146]:
novy_df.groupby('COUNTRY')

In [147]:
zkraceny_novy_df = novy_df[['COUNTRY', 'OFFERED_CAPACITY_PRICE_NEW']]

<br>

Opatrně na seskupování, tak aby data dávala nějaký smysl:

In [148]:
zkraceny_novy_df.groupby('COUNTRY').max()

,OFFERED_CAPACITY_PRICE_NEW
COUNTRY,
AT,59.1100
BE,46.8350
CH,61.4750
CZ,50.0000
DE,62.2500
DK,26.2725
FR,50.0000
NL,18.0000
SI,25.0000


In [149]:
dalsi_zkraceny_novy_df = novy_df[['COUNTRY', 'PRODUCT', 'OFFERED_CAPACITY_PRICE_NEW', 'SETTLEMENTCAPACITY_PRICE_NEW']]

In [150]:
dalsi_zkraceny_novy_df

,COUNTRY,PRODUCT,OFFERED_CAPACITY_PRICE_NEW,SETTLEMENTCAPACITY_PRICE_NEW
0,DE,NEGPOS_00_04,0.0000,5.500
1,DE,NEGPOS_00_04,0.0000,5.500
2,DE,NEGPOS_00_04,0.0000,5.500
3,DE,NEGPOS_00_04,0.0000,5.500
4,DE,NEGPOS_00_04,0.0000,5.500
...,...,...,...,...
1681,AT,NEGPOS_20_24,13.9000,14.535
1682,CH,NEGPOS_20_24,13.9500,14.535
1683,BE,NEGPOS_20_24,14.2675,14.535
1684,CH,NEGPOS_20_24,14.4000,14.535


In [151]:
vystup = dalsi_zkraceny_novy_df.pivot_table(values='OFFERED_CAPACITY_PRICE_NEW',
                                   index='COUNTRY', columns='PRODUCT').round(1).unstack()

In [152]:
vystup.index

MultiIndex([('NEGPOS_00_04', 'AT'),
            ('NEGPOS_00_04', 'BE'),
            ('NEGPOS_00_04', 'CH'),
            ('NEGPOS_00_04', 'CZ'),
            ('NEGPOS_00_04', 'DE'),
            ('NEGPOS_00_04', 'DK'),
            ('NEGPOS_00_04', 'FR'),
            ('NEGPOS_00_04', 'NL'),
            ('NEGPOS_00_04', 'SI'),
            ('NEGPOS_04_08', 'AT'),
            ('NEGPOS_04_08', 'BE'),
            ('NEGPOS_04_08', 'CH'),
            ('NEGPOS_04_08', 'CZ'),
            ('NEGPOS_04_08', 'DE'),
            ('NEGPOS_04_08', 'DK'),
            ('NEGPOS_04_08', 'FR'),
            ('NEGPOS_04_08', 'NL'),
            ('NEGPOS_04_08', 'SI'),
            ('NEGPOS_08_12', 'AT'),
            ('NEGPOS_08_12', 'BE'),
            ('NEGPOS_08_12', 'CH'),
            ('NEGPOS_08_12', 'CZ'),
            ('NEGPOS_08_12', 'DE'),
            ('NEGPOS_08_12', 'DK'),
            ('NEGPOS_08_12', 'FR'),
            ('NEGPOS_08_12', 'NL'),
            ('NEGPOS_08_12', 'SI'),
            ('NEGPOS_12_16',

In [153]:
import numpy as np

In [154]:
agregacni_predpis = {'OFFERED_CAPACITY_PRICE_NEW': np.median}

In [155]:
dalsi_zkraceny_novy_df.pivot_table(values='OFFERED_CAPACITY_PRICE_NEW',
                                   index='COUNTRY', columns='PRODUCT',
                                   aggfunc=agregacni_predpis).round(1)

PRODUCT,NEGPOS_00_04,NEGPOS_04_08,NEGPOS_08_12,NEGPOS_12_16,NEGPOS_16_20,NEGPOS_20_24
COUNTRY,,,,,,
AT,2.4,4.3,2.8,2.7,2.7,2.7
BE,3.0,3.0,3.6,4.0,3.1,4.2
CH,4.0,7.1,32.5,33.2,16.2,7.4
CZ,0.8,1.0,1.0,1.0,1.0,1.0
DE,0.6,0.2,0.2,0.2,0.2,0.2
DK,0.2,0.2,0.2,2.5,0.2,0.2
FR,0.6,0.6,0.6,0.6,0.5,0.6
NL,1.6,2.1,2.1,0.0,2.1,1.8
SI,NaN,NaN,25.0,16.5,25.0,NaN


---


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.odqbr-09dxxBQjF7x7NyBAHaII%26pid%3DApi&f=1&ipt=5b4a03c1dd38b52108237102ebb350595a5d6ea6a50572f5897ca89fbf5b5445&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

<br>

## Časové řady

---

Část *frameworku* byla vyvinuta za účelem **finančního modelování**.

Proto je více než dobře vybavena sadou nástrojů, které umí pracovat **s daty, časem a časovými objekty**.

<br>

Uplatnění:
* **Analýza a predikce**: Časové řady umožňují analyzovat vývoj hodnot v čase a předpovídat budoucí trendy nebo sezónní změny. To je klíčové v oblastech jako finance, ekonomika, prodej, počasí a mnoho dalších.

* **Efektivní manipulace s časem**: Pandas poskytuje nástroje pro efektivní práci s časovými řadami, což zjednodušuje úkoly, jako je agregace, interpolace nebo časové posuny. Díky integrovaným funkcím můžete snadno pracovat s daty různých časových frekvencí a konvertovat mezi nimi.

* **Časově závislé analýzy**: Časové řady umožňují provádět časově závislé analýzy, jako je detekce změn v trendech, identifikace sezónních vlivů nebo identifikace časových závislostí mezi proměnnými.

* **Flexibilní indexace**: Pandas podporuje flexibilní indexaci časových řad, což usnadňuje filtrování, řazení a výběr dat na základě časových značek nebo rozsahů.

* **Kompatibilita s dalšími knihovnami**: Pandas je kompatibilní s mnoha dalšími knihovnami pro analýzu časových řad, jako je statsmodels nebo scikit-learn. To usnadňuje integraci a rozšíření vašich analýz s pokročilejšími metodami a algoritmy.
    
<br>

Jde například o údaje typu:
* *timestampy*, údaj odkazující na konkrétní **časový okamžik** (např. `26. července 2024 v 14:36 hod.`),
* *časové intervaly*, tedy období odkazují na délku času **mezi konkrétním začátkem a koncem** (např. intervaly ze dne na den),
* *time delta* objekty, tedy přesné délky času (např. 22,22 sekundy).

#### Data a čas v Pythonu

---

Standardní výbavou Pythonu jsou knihovny `datetime`:

In [156]:
from datetime import datetime

In [157]:
datetime(year=2023, month=4, day=5)

datetime.datetime(2023, 4, 5, 0, 0)

<br>

Nebo knihovna `dateutil` pro parsování datových typů z různých stringových zadání:

In [158]:
from dateutil import parser

In [159]:
date = parser.parse("26th of november, 2024")

In [160]:
date

datetime.datetime(2024, 11, 26, 0, 0)

Kde pomocí metody `strftime` můžeš vypsat den:

In [161]:
date.strftime("%A")

'Tuesday'

#### Data a čas v numpy

---

Některé nedostatky uvnitř knihoven `datetime` a `dateutil` vedli ke vzniku sady nástrojů.

Tyto doplňky vznikly pod hlavičkou knihovny `numpy`.

In [162]:
import numpy as np
from numpy import array, arange

In [163]:
date = array('2023-04-05', dtype=np.datetime64)

In [164]:
date

array('2023-04-05', dtype='datetime64[D]')

<br>

Pokud potřebuješ pole následujících 7 dní:

In [165]:
date + arange(7)

array(['2023-04-05', '2023-04-06', '2023-04-07', '2023-04-08',
       '2023-04-09', '2023-04-10', '2023-04-11'], dtype='datetime64[D]')

<br>

Vzhledem k jednotnému datovu typu v poli pro **numpy** `datetime64` může tento typ operace
provádět mnohem rychleji, než přímo v Pythonu `datetime` objekty, zejména když objekty nabývají na velikosti.

#### Data a čas v pandách

---

Jde o kombinace objektů z obou předchozích podkapitol.

Ty dávají dohromady to nejlepší prostředky pro zacházení s časem.

In [166]:
from pandas import to_datetime

In [167]:
date = to_datetime("5th of April, 2023")

In [168]:
date

Timestamp('2023-04-05 00:00:00')

In [169]:
date.strftime("%A")

'Wednesday'

## Časové řady

---

V podstatě jde o hlavní nástroj, který tato knihovna dovede nabídnout.

#### Indexování časem
---
`DatetimeIndex` obsahuje časové značky (*timestamp*), které jsou uloženy ve formátu `datetime64` s nanosekundovou přesností.

Tento objekt umožňuje efektivní práci s časovými řadami a poskytuje mnoho funkcí pro manipulaci s daty a časy.

Výhody práce s `DatetimeIndex` objektem:
* Časové zóny,
* frekvence,
* časově závislé selekce,
* atributy časových značek,
* operace s časem.

In [170]:
from pandas import DatetimeIndex, Series, to_datetime

In [171]:
datumy = ["2023-04-05", "2022-04-05", "2021-04-05", "2020-04-05"]

In [172]:
indexy = DatetimeIndex(datumy)

In [173]:
hodnoty = [to_datetime(den).strftime("%A") for den in datumy]

In [174]:
hodnoty

['Wednesday', 'Tuesday', 'Monday', 'Sunday']

In [175]:
hodnoty_sl = Series(hodnoty, index=indexy)

In [176]:
hodnoty_sl

2023-04-05    Wednesday
2022-04-05      Tuesday
2021-04-05       Monday
2020-04-05       Sunday
dtype: object

In [177]:
hodnoty_sl.index

DatetimeIndex(['2023-04-05', '2022-04-05', '2021-04-05', '2020-04-05'], dtype='datetime64[ns]', freq=None)

In [178]:
type(indexy)

pandas.core.indexes.datetimes.DatetimeIndex

<br>

Můžeš vytvořit selektivní proces na základě datumu pro index:

In [179]:
hodnoty_sl[:'2021-04-05']

2021-04-05    Monday
2020-04-05    Sunday
dtype: object

<br>

## Základní objekty

---

Mezi základní objekty pro práci s časem patří:
* `Timestamp` typ (související struktura Indexu `DatetimeIndex`,
* `Period` typ (.. `PeriodIndex`),
* `Timedelta` typ (.. `TimedeltaIndex`).

#### Timestamp & DatetimeIndex

---

Nejčastější datové typy, které lze vyvolat přímo, ovšem běžnější je pracovat s funkcí `to_datetime`.

Funkce `to_datetime` umí parsovat různé stringové formáty.

<br>

### Práce s jedním datumem

---

In [180]:
datum_objekt = to_datetime("05/04/2023")  # DD/MM/RRRR, 5. duben 2023

<br>

Opatrně na specifický formát datumu:

In [181]:
datum_objekt                              # MM/DD/RRRR

Timestamp('2023-05-04 00:00:00')

In [182]:
type(datum_objekt)

pandas._libs.tslibs.timestamps.Timestamp

In [183]:
datum_objekt.month_name()

'May'

Pokud do funkce `to_datetime` vložíš jedinou hodnotu, vrací objekty typu `Timestamp`.

<br>

#### Specifický formát

---

Pokud se *parser* ztratí nebo tvoje zadání neodpovídá jeho vyhotovení:

In [184]:
to_datetime("12-11-2010 00:00", format="%d-%m-%Y %H:%M")

Timestamp('2010-11-12 00:00:00')

In [185]:
naformatovany_datum = to_datetime("05/04/2023", format='%d/%m/%Y')

In [186]:
naformatovany_datum.month_name()

'April'

In [187]:
naformatovany_datum.day_name()

'Wednesday'

Nespoléhat na defaultní nastavení funkce `to_datetime`, ale umět si formát upravit podle svých potřeb.

<br>

### Práce s několika datumy

---

In [188]:
datumy = to_datetime([datetime(2023, 4, 5),
                      "5th of April 2023",
                      "2023-Apr-5",
                      "05-04-2023",
                      "20230405"])

In [189]:
datumy

DatetimeIndex(['2023-04-05', '2023-04-05', '2023-04-05', '2023-05-04',
               '2023-04-05'],
              dtype='datetime64[ns]', freq=None)

In [190]:
type(datumy)

pandas.core.indexes.datetimes.DatetimeIndex

Zatímco pole hodnot s datumy, které funkce `to_datetime` zpracuje vrací objekt typu `DatetimeIndex`.

<br>

Objekt `DatetimeIndex` potom můžeš konvertovat na `PeriodIndex` pomocí *metody* `to_period`:

In [191]:
# datumy.to_period?

<br>

Tato metoda je užitečná, pokud chcete převést časové řady **na určité časové období**.

Když budeš třeba potřebovat převést hodnoty **z denních dat na měsíční data**.

<br>

#### Denní data

---

In [192]:
datumy.to_period(freq="D")

PeriodIndex(['2023-04-05', '2023-04-05', '2023-04-05', '2023-05-04',
             '2023-04-05'],
            dtype='period[D]')

<br>

#### Měsíční data

---

In [193]:
from pandas import DataFrame
import pandas as pd

In [194]:
datumy.to_period(freq="M")

PeriodIndex(['2023-04', '2023-04', '2023-04', '2023-05', '2023-04'], dtype='period[M]')

<br>

V rámci periody jsem schopen u časových řad nastavovat, nebo měnit časové intervaly (tzn. denní, měsíční).

In [195]:
# data = {'date': pd.date_range(start='2022-01-01', periods=10, freq='ME'),
#         'sales': [100, 102, 105, 107, 110, 112, 115, 117, 120, 122]}

<br>

Varianta frekvence `ME` lze použít ve verzi pandas 2.2.

Pro starší varianty je nutné pracovat s argumentem `M`.

In [196]:
data = {'date': pd.date_range(start='2022-01-01', periods=10, freq='M'),
        'sales': [100, 102, 105, 107, 110, 112, 115, 117, 120, 122]}

In [197]:
data_df = DataFrame(data)

In [198]:
data_df

,date,sales
0,2022-01-31,100
1,2022-02-28,102
2,2022-03-31,105
3,2022-04-30,107
4,2022-05-31,110
5,2022-06-30,112
6,2022-07-31,115
7,2022-08-31,117
8,2022-09-30,120
9,2022-10-31,122


<br>

Index původní tabulky je nastavený implicitně jako interval `0-n` .

In [199]:
data_df.set_index('date', inplace=True)

In [200]:
data_df

,sales
date,
2022-01-31,100
2022-02-28,102
2022-03-31,105
2022-04-30,107
2022-05-31,110
2022-06-30,112
2022-07-31,115
2022-08-31,117
2022-09-30,120


In [201]:
period_df = data_df.to_period(freq='M')

In [202]:
period_df

,sales
date,
2022-01,100
2022-02,102
2022-03,105
2022-04,107
2022-05,110
2022-06,112
2022-07,115
2022-08,117
2022-09,120


<br>

### TimedeltaIndex

---

Jde o rozdíl v jednotkách času.

Nejčastěji se s tímto objektem setkáš, pokud potřebuješ získat **rozdíl mezi dvěma datumy**.

In [203]:
from datetime import datetime

from pandas import to_datetime

In [204]:
import pandas as pd

In [205]:
specialni_datumy = to_datetime([datetime(2022, 4, 5),
                                "5th of April 2021",
                                "2020-Apr-5",
                                "05-04-2019",
                                "20180405"])

In [206]:
specialni_datumy

DatetimeIndex(['2022-04-05', '2021-04-05', '2020-04-05', '2019-05-04',
               '2018-04-05'],
              dtype='datetime64[ns]', freq=None)

In [207]:
datumy

DatetimeIndex(['2023-04-05', '2023-04-05', '2023-04-05', '2023-05-04',
               '2023-04-05'],
              dtype='datetime64[ns]', freq=None)

In [208]:
datumy - specialni_datumy

TimedeltaIndex(['365 days', '730 days', '1095 days', '1461 days', '1826 days'], dtype='timedelta64[ns]', freq=None)

<br>

Rozdíl mezi datumy můžeš počítat i pro jednotlivé `Timestamp` hodnoty:

In [209]:
datumy[0]

Timestamp('2023-04-05 00:00:00')

In [210]:
specialni_datumy[0]

Timestamp('2022-04-05 00:00:00')

In [211]:
datumy[0] - specialni_datumy[0]

Timedelta('365 days 00:00:00')

In [212]:
datovy_objekt = to_datetime('2022-04-05')

In [213]:
datovy_objekt.tzname()

Údaj o časovém pásmu aktuálně chybí.

<br>

In [214]:
aktualni_cas = pd.Timestamp('2017-01-01T12')

In [215]:
aktualni_cas

Timestamp('2017-01-01 12:00:00')

<br>

Pokud chybí v objektu informace o pásmu, je možnost ji doplnit:

In [216]:
aktualni_cas = aktualni_cas.tz_localize('CET')

Metoda `tz_localize` nastaví nové hodnoty časového pásma k časovému údaji.

<br>

In [217]:
aktualni_cas

Timestamp('2017-01-01 12:00:00+0100', tz='CET')

Pokud potřebuješ hodnotu upravit, použiješ metodu `tz_convert`:

In [218]:
aktualni_cas.tz_convert('Europe/Prague')

Timestamp('2017-01-01 12:00:00+0100', tz='Europe/Prague')

In [219]:
aktualni_cas.tzname()

'CET'

Metoda `tzname` potom hodnotu zobrazí.

<br>

#### Vybereme datum v zimním čase a v letním čase

---

In [2]:
import pandas as pd

In [3]:
zimni = pd.Timestamp("2024-01-15 12:00:00", tz="Europe/Prague")
letni = pd.Timestamp("2024-07-15 12:00:00", tz="Europe/Prague")

#### Převod na UTC

---

In [4]:
zimni_utc = zimni.tz_convert("UTC")
letni_utc = letni.tz_convert("UTC")

In [5]:
print("Zimní čas (CET):", zimni)
print("Zimní čas v UTC:", zimni_utc)

Zimní čas (CET): 2024-01-15 12:00:00+01:00
Zimní čas v UTC: 2024-01-15 11:00:00+00:00


In [6]:
print("Letní čas (CEST):", letni)
print("Letní čas v UTC:", letni_utc)

Letní čas (CEST): 2024-07-15 12:00:00+02:00
Letní čas v UTC: 2024-07-15 10:00:00+00:00


#### Rozdíl mezi nimi

---

In [8]:
rozdil = (letni.utcoffset() - zimni.utcoffset()).total_seconds() / 3600
print("Rozdíl v hodinách mezi zimním a letním časem:", rozdil)

Rozdíl v hodinách mezi zimním a letním časem: 1.0


<br>

#### 🧠 CVIČENÍ 🧠, procvič si funkci `to_datetime`

---

In [220]:
# Převeď datumy na formát RRRR-MM-DD
data = {"Name": ["Event A", "Event B", "Event C"],
        "Date": ["21-11-2024", "05-12-2024", "15-01-2025"]}

In [221]:
data_df = DataFrame(data, index=data['Date'])

In [222]:
data_df

,Name,Date
21-11-2024,Event A,21-11-2024
05-12-2024,Event B,05-12-2024
15-01-2025,Event C,15-01-2025


In [223]:
data_df['Date'] = to_datetime(data_df['Date'], format='%d-%m-%Y')

In [224]:
data_df

,Name,Date
21-11-2024,Event A,2024-11-21
05-12-2024,Event B,2024-12-05
15-01-2025,Event C,2025-01-15


<details>
    <summary>▶️ Řešení</summary>
    
```python
import pandas as pd

data = {
    "Name": ["Event A", "Event B", "Event C"],
    "Date": ["21-11-2024", "05-12-2024", "15-01-2025"],
}


df = pd.DataFrame(data)


def convert_dates(df):
    df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")
    return df

converted_df = convert_dates(df)
print(converted_df)
print(converted_df.dtypes)
```
</details>

<br>

## Funkce `date_range`

---

Aby bylo zadání řady (sekvence) dat pohodlnější, vyzkoušej funkci `date_range`.

Obdobně potom pracují související funkce:
* `date_range`, timestampy,
* `period_range`, periody,
* `timedelta_range`, pro delty.

In [225]:
from pandas import date_range

<br>

### Počet period

---

In [226]:
datumy_rada = date_range(start="01-01-1992", periods=5, freq='10min15s')

<br>

Specifikace hodnoty frekvence vychází vždy z tabulky [offsetů](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).

In [227]:
datumy_rada

DatetimeIndex(['1992-01-01 00:00:00', '1992-01-01 00:10:15',
               '1992-01-01 00:20:30', '1992-01-01 00:30:45',
               '1992-01-01 00:41:00'],
              dtype='datetime64[ns]', freq='615S')

<br>

## Frekvence

---

In [228]:
datumy_mesicne_ind = date_range("01-01-1992", periods=12, freq="M")

In [229]:
datumy_mesicne_ind

DatetimeIndex(['1992-01-31', '1992-02-29', '1992-03-31', '1992-04-30',
               '1992-05-31', '1992-06-30', '1992-07-31', '1992-08-31',
               '1992-09-30', '1992-10-31', '1992-11-30', '1992-12-31'],
              dtype='datetime64[ns]', freq='M')

<br>

### Hodinové periody

---

In [230]:
from pandas import timedelta_range

In [231]:
hodinove_ind = timedelta_range(0, periods=12, freq='H')

In [232]:
hodinove_ind

TimedeltaIndex(['0 days 00:00:00', '0 days 01:00:00', '0 days 02:00:00',
                '0 days 03:00:00', '0 days 04:00:00', '0 days 05:00:00',
                '0 days 06:00:00', '0 days 07:00:00', '0 days 08:00:00',
                '0 days 09:00:00', '0 days 10:00:00', '0 days 11:00:00'],
               dtype='timedelta64[ns]', freq='H')

<br>

### Tabulka frekvencí

---

Časovou paletou frekvencí, kterou framework `pandas` nabízí je tato tabulka:

| String | Popisek |
| :-: | :- |
| `D` | kalendářní den |
| `W` | týden |
| `M` | konec měsíce |
| `Q` | konec čtvrtletí |
| `A` | konec roku |
| `H` | hodiny |
| `T` | minuty |
| `S` | vteřiny |
| `B` | pracovní den |
| `BM` | konec pracovního měsíce |
| `BQ` | konec pracovního čtvrtletí |

#### Netradiční frekvence

---

In [233]:
timedelta_range(start='1H', periods=5, freq="1H15T")

TimedeltaIndex(['0 days 01:00:00', '0 days 02:15:00', '0 days 03:30:00',
                '0 days 04:45:00', '0 days 06:00:00'],
               dtype='timedelta64[ns]', freq='75T')

<br>

### Resampling

---

*Resampling* nebo také *převzorkování* je proces, který upravuje frekvenci časové řady.

*Resampling* se obvykle používá **pro snížení frekvence** (z hodinových dat na denní data).

*Resampling* zahrnuje **agregaci dat**.

In [234]:
import pandas as pd
import numpy as np
from pandas import DataFrame, date_range

<br>

V `pandas` se k tomu používá metoda `resample`, která má jako parametr novou frekvenci, na kterou chcete data převést:

In [235]:
data = {'date': pd.date_range(start='2022-01-01', periods=60, freq='D'),
        'sales': np.random.randint(10, 100, size=60)}

In [236]:
data_df = pd.DataFrame(data)

In [237]:
data_df.head(7)

,date,sales
0,2022-01-01,42
1,2022-01-02,98
2,2022-01-03,86
3,2022-01-04,85
4,2022-01-05,91
5,2022-01-06,84
6,2022-01-07,83


Dostupné sloupce:

In [238]:
data_df.columns

Index(['date', 'sales'], dtype='object')

Dostupné indexy:

In [239]:
data_df.index

RangeIndex(start=0, stop=60, step=1)

<br>

Nastavení nového sloupce pro Index:

In [240]:
data_df.set_index('date', inplace=True)

In [241]:
data_df.head()

,sales
date,
2022-01-01,42
2022-01-02,98
2022-01-03,86
2022-01-04,85
2022-01-05,91


<br>

Pomocí metody `resample` současně hodnoty agregujeme a snížíme frekvenci:

In [242]:
pd.__version__

'2.0.3'

In [243]:
mesicni_data_df = data_df.resample('M').sum()  # 'M'/'ME' - end of the month

In [244]:
mesicni_data_df

,sales
date,
2022-01-31,1658
2022-02-28,1497
2022-03-31,83


In [245]:
data_df.tail()

,sales
date,
2022-02-25,38
2022-02-26,47
2022-02-27,26
2022-02-28,68
2022-03-01,83


<br>

Upravit původní vzorek dat na zamýšlenou frekvenci, tedy `W`:

In [246]:
tydenni_data_df = data_df.resample('W').mean().round(1)

In [247]:
tydenni_data_df

,sales
date,
2022-01-02,70.0
2022-01-09,72.3
2022-01-16,50.0
2022-01-23,47.6
2022-01-30,40.9
2022-02-06,45.7
2022-02-13,56.0
2022-02-20,56.9
2022-02-27,51.7


In [248]:
from pandas import to_datetime

In [249]:
to_datetime('2022-01-02').day_name()

'Sunday'

<br>

### Zpřesňování

---

Zahrnuje **zvýšení frekvence časových řad**.

Přičemž se nově vytvořené hodnoty obvykle interpolují nebo doplňují nějakou konstantou.

V pandas se k zpřesňování používá metoda `asfreq`, která má jako parametr **novou frekvenci**, na kterou chcete data převést.

In [250]:
data = {'date': pd.date_range(start='2022-01-01', end='2022-03-01', freq='D'),
        'sales': np.random.randint(10, 150, size=60)}  # Opatrně na délky sekvencí

In [251]:
dalsi_data_df = pd.DataFrame(data)

In [252]:
dalsi_data_df.head()

,date,sales
0,2022-01-01,99
1,2022-01-02,70
2,2022-01-03,134
3,2022-01-04,35
4,2022-01-05,34


In [253]:
dalsi_data_df.set_index('date', inplace=True)

In [254]:
dalsi_data_df.head(10)

,sales
date,
2022-01-01,99
2022-01-02,70
2022-01-03,134
2022-01-04,35
2022-01-05,34
2022-01-06,146
2022-01-07,44
2022-01-08,101
2022-01-09,18


In [255]:
denni_data = dalsi_data_df.asfreq('H')

<br>

Pokud nezadáš hodnotu pro výplňovací parametr, uvidíš prázdné hodnoty `NaN`:

In [256]:
denni_data.head(25)

,sales
date,
2022-01-01 00:00:00,99.0
2022-01-01 01:00:00,NaN
2022-01-01 02:00:00,NaN
2022-01-01 03:00:00,NaN
2022-01-01 04:00:00,NaN
2022-01-01 05:00:00,NaN
2022-01-01 06:00:00,NaN
2022-01-01 07:00:00,NaN
2022-01-01 08:00:00,NaN


Metoda `asfreq` bere jako parametr novou frekvenci, na kterou chceme data převést.

V případě výše jde o frekvenci `H`, což znamená **hodinová data**.

Protože při zpřesňování se **vytvoří chybějící hodnoty** (v důsledku zvýšení frekvence), je potřeba zvolit metodu pro jejich doplnění.

In [257]:
denni_data_df = dalsi_data_df.asfreq('H', method='ffill')  # ffill, bfill

In [258]:
denni_data_df.head(26)

,sales
date,
2022-01-01 00:00:00,99
2022-01-01 01:00:00,99
2022-01-01 02:00:00,99
2022-01-01 03:00:00,99
2022-01-01 04:00:00,99
2022-01-01 05:00:00,99
2022-01-01 06:00:00,99
2022-01-01 07:00:00,99
2022-01-01 08:00:00,99


Výš jde o metodu `forward fill` (parametr `method='ffill'`), která kopíruje předchozí hodnotu pro výplň chybějících hodnot.

Další možností je použít metodu `'back fill'` (parametr `method='bfill'`), která kopíruje následující hodnotu pro výplň chybějících hodnot.

In [259]:
denni_data = dalsi_data_df.asfreq('H', method='bfill')

In [260]:
denni_data.head(25)

,sales
date,
2022-01-01 00:00:00,99
2022-01-01 01:00:00,70
2022-01-01 02:00:00,70
2022-01-01 03:00:00,70
2022-01-01 04:00:00,70
2022-01-01 05:00:00,70
2022-01-01 06:00:00,70
2022-01-01 07:00:00,70
2022-01-01 08:00:00,70


<br>

Pokud mi nevyhovují metody `bfill` a `ffill`, tak můžu nově vzniklé záznamy vyplnit konstantou `fill_value`.

<br>

**🧠 CVIČENÍ 🧠, procvič si časové řady**

Analyzuj prodeje produktu ve fiktivní společnosti během jednoho roku a zjisti následující:
1. Celkový prodej za každý měsíc.
2. Průměrný prodej za každý den v týdnu.
3. Denní prodej za poslední týden.

In [261]:
import pandas as pd
import numpy as np

In [262]:
denni_datumy = pd.date_range(start='2022-01-01',
                             end='2022-12-31',
                             freq='D')
prodeje = np.random.randint(10, 100, size=(len(denni_datumy),))

In [263]:
df_prodeje = pd.DataFrame({'datumy': denni_datumy, 'prodeje': prodeje})

In [264]:
df_prodeje.set_index('datumy', inplace=True)

#### Celkový prodej za měsíc

---

In [265]:
df_prodeje.head()

,prodeje
datumy,
2022-01-01,86
2022-01-02,63
2022-01-03,66
2022-01-04,67
2022-01-05,35


In [266]:
r1 = df_prodeje.resample('M').sum().to_period(freq='M')

#### Upravit formát data na RRRR-MM

---

In [267]:
from pandas import to_datetime

In [270]:
df_prodeje

,prodeje
datumy,
2022-01-01,86
2022-01-02,63
2022-01-03,66
2022-01-04,67
2022-01-05,35
...,...
2022-12-27,29
2022-12-28,37
2022-12-29,92


In [273]:
df_prodeje.index = df_prodeje.index.strftime('%Y/%m')

In [275]:
df_prodeje.head()

,prodeje
datumy,
2022/01,86
2022/01,63
2022/01,66
2022/01,67
2022/01,35


<details>
    <summary>▶️ Řešení</summary>
    
```python
monthly_sales = df.resample('M').sum()
print("Celkový prodej za každý měsíc:")
print(monthly_sales)

df_prodeje['weekday'] = df_prodeje.index.weekday
average_weekday_sales = df_prodeje.groupby('weekday')['prodeje'].mean()
print("\nPrůměrný prodej za každý den v týdnu:")
print(average_weekday_sales)

last_week_sales = df.loc['2022-12-25':, 'sales']
print("\nDenní prodej za poslední týden:")
print(last_week_sales)
```
</details>

---